Requirements:
- [Get started](../01-Day_1/02-Get_started.ipynb)
- [Data Management](../01-Day_1/04-Data_management.ipynb)

# Loop tiling

Nested loops often reuse the same data across their iterations and keeping the working set inside the caches can improve performance.
Tiling is a partitioning method of the loops into blocks. It reorders the loops so that each block will repeatedly hit the cache.
A first usage restriction will thus be on the loops' nature itself: not all loops can benefit from tiling, only the ones that will reuse data while showing a poor data locality, thus leading to frequent cache misses.

<img src="../../pictures/tiles.png"/>

OpenACC allows to improve data locality inside loops with the dedicated _tile_ clause. 
It specifies the compiler to split each loop in the nest into 2 loops, with an outer set of tile loops and an inner set of element loops.

## Syntax

The tile clause may appear with the _loop_ directive for nested loops.
For _N_ nested loops, the tile clause can take _N_ arguments. The first one being the size of the inner loop of the nest, the last one being the size of the outer loop.

```c
#pragma acc loop tile(32,32)
for(int i = 0 ; i < size_i ; ++i)
{
    for(int j = 0 ; j < size_j ; ++j)
    {
        // A Fabulous calculation
    }
}    
```

## Restrictions

- the tile size (corresponding to the product of the arguments of the tile clause) can be up to 1024
- for better performance the size for the inner loop is a power of 2 (best with 32 to fit a cuda warp)
- if the vector clause is specified, it is then applied to the element loop
- if the gang clause is specified, it is then applied to the tile loop
- the worker clause is applied to the element loop only if the vector clause is not specified

## Example

In the following example tiling is used to transpose a matrix.

In [ ]:
%%idrrun -a --time
//  examples/C/Loop_tiling_example.c
#include <stdio.h>
int main(void)
{ 
    int ny = 10000;
    int nx = 10000;
    int idxA, idxB;
    int a[nx*ny], b[nx*ny];

// Structured data region
    #pragma acc data copyout(a, b)
    {
        #pragma acc parallel loop present(a, b)
        for(int i=0; i<nx*ny; ++i)
            a[i] = i;
           
        #pragma acc parallel loop present(a, b) tile(32,32)
        for (int i=0; i<nx; ++i){
            for (int j=0; j<ny; ++j){
                idxB = j + i*nx;
                idxA = i + j*ny;
                b[idxB] = a[idxA];
            }
        }
    }
 
    fprintf(stderr,"A[%d]=%d , A[%d]=%d\n",1+2*nx,a[1+2*nx],2+1*ny,a[2+1*ny] );
    fprintf(stderr,"A[%d]=%d , A[%d]=%d\n",1+2*nx,b[1+2*nx],2+1*ny,b[2+1*ny] );
}

## Exercise

In this exercise, you will try to accelerate the numerical resolution of the 2D Laplace's equation with tiles. You can see that tiles parameter should be chosen wisely in order not to deteriorate performance.

In [ ]:
%%idrrun -a
//  examples/C/Loop_tiling_exercise.c
#include<stdio.h>
#include<stdlib.h>
#include<math.h>
#include<openacc.h>

int main(void) {
    int    nx = 20000;
    int    ny = 10000;
    int    idx; 
    double T[nx*ny], T_new[nx*ny];
    double erreur;

    for(int i=1; i<nx-1; ++i) {
        for(int j=1; j<ny-1; ++j) {
        T[i*ny+j]     = 0.0;
        T_new[i*ny+j] = 0.0;
        }
    }

    for(int i=0; i<nx; ++i){
        T[i*ny     ] = 100.0;
        T[i*ny+ny-1] = 0.0;
    }

    for(int j=0; j<ny; ++j){
        T[j]           = 0.0;
        T[(nx-1)*ny+j] = 0.0;
    }

    // add acc directive
    for (int it = 0; it<10000; ++it){
        erreur = 0.0;	
        // add acc directive 
        for (int i=1; i<nx-1; ++i) {
            for (int j=1; j<ny-1; ++j) {
                idx = i*(ny)+j;
                T_new[idx] = 0.25*(T[idx+ny]+T[idx-ny] + T[idx+1]+T[idx-1]);
                erreur = fmax(erreur, fabs(T_new[idx]-T[idx]));
            }
         }
        if(it%100 == 0) fprintf(stderr,"it: %d, erreur: %e\n",it,erreur);

        // add acc directive
        for (int i=1; i<nx-1; ++i) {
            for (int j=1; j<ny-1; ++j) {
                T[i*(ny)+j] =  T_new[i*(ny)+j];
            }
        }
    }
    return 0;
}

## Solution

In [ ]:
%%idrrun -a
//  examples/C/Loop_tiling_solution.c
#include<stdio.h>
#include<stdlib.h>
#include<math.h>
#include<openacc.h>

int main(void) {
    int    nx = 20000;
    int    ny = 10000; 
    int    idx;    
    double T[nx*ny], T_new[nx*ny];
    double erreur;

    for(int i=1; i<nx-1; ++i) {
        for(int j=1; j<ny-1; ++j) {
        T[i*ny+j]     = 0.0;
        T_new[i*ny+j] = 0.0;
        }
    }

    for(int i=0; i<nx; ++i){
        T[i*ny     ] = 100.0;
        T[i*ny+ny-1] = 0.0;
    }

    for(int j=0; j<ny; ++j){
        T[j]           = 0.0;
        T[(nx-1)*ny+j] = 0.0;
    }

#pragma acc data copy(T) create(T_new)    
{
    for (int it = 0; it<10000; ++it){
        erreur = 0.0;	
        #pragma acc parallel loop tile (32,32) reduction(max:erreur)
        for (int i=1; i<nx-1; ++i) {
            for (int j=1; j<ny-1; ++j) {
                idx = i*(ny)+j;
                T_new[idx] = 0.25*(T[idx+ny]+T[idx-ny] + T[idx+1]+T[idx-1]);
                erreur = fmax(erreur, fabs(T_new[idx]-T[idx]));
            }
        }
        if(it%100 == 0) fprintf(stderr,"it: %d, erreur: %e\n",it,erreur);

        #pragma acc parallel loop 
        for (int i=1; i<nx-1; ++i) {
            for (int j=1; j<ny-1; ++j) {
                T[i*(ny)+j] =  T_new[i*(ny)+j];
            }
        }
    }
}
    return 0;
}